# Models for AHR and MMP endpoints
Here the model training for the two models will done. 

In [ ]:
import pandas as pd
import numpy as np

import pickle

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import root_mean_squared_error,mean_absolute_error
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import scale

import optuna

In [ ]:
#Load the datasets
with open('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Code/Tox21 comparison/2025-05-12_tox21_ahr_mmp_available_compounds_all_sources_UPDATED.pkl', 'rb') as f:
    tox21 = pickle.load(f)

with open('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Data/Tox21/2025-03-12_Tox21_remade_fp.pkl', 'rb') as f:
    tox21_fp = pickle.load(f)

ms_data = pd.read_excel('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Experimental/Experimental_work/2025-04-25_active_mixes_ms.xlsx')

In [ ]:
tox21

In [ ]:
tox21_fp.sirius5_fp.notna().sum()

In [ ]:
expanded_fp = tox21_fp['sirius5_fp'].apply(pd.Series)

In [ ]:
expanded_fp.shape

In [ ]:
expanded_fp.iloc[:, 0].isna().sum()

In [ ]:
tox21_fp['sirius5_fp'].isna().sum()

In [ ]:
tox21_fp

In [ ]:
expanded_fp = tox21_fp['sirius5_fp'].apply(pd.Series)

In [ ]:
tox21_fp.sirius5_fp.isna().sum()

In [ ]:
tox21

In [ ]:
ms_data = ms_data[['name', 'SMILES', 'Found with UNIFI']]
ms_data = ms_data.rename(columns={'Found with UNIFI': 'ionization_experimental'})
ms_data = ms_data.dropna(subset=['name'])

ms_data

In [ ]:
tox21 = pd.merge(tox21, ms_data[['ionization_experimental', 'SMILES']], how='left', on='SMILES')

tox21[tox21['ionization_experimental'].notna()]

In [ ]:
tox21 = tox21.merge(tox21_fp[['InChIKey14', 'sirius5_fp']], on='InChIKey14', how='left')
tox21[tox21.sirius5_fp.isna()]

In [ ]:
tox21 = tox21[tox21.sirius5_fp.notna()]

tox21

In [ ]:
# with open('tox21_all_sources_fp.pkl', 'wb') as f:
#     pickle.dump(tox21, f)

### Training, testing and evaluation data
The training and testing data will be a 9:1 slit between the compounds which are not available in the library, Iris data, Isabells data, SIRIUS training data or the experimental acquisition data. The evaluation will be the others mentioned combined. 

In [ ]:
with open('tox21_all_sources_fp.pkl', 'rb') as f:
    tox21 = pickle.load(f)

In [ ]:
tox21

In [ ]:
#Training data includes all compounds which doesn't have any experimental data, or if they were not ionized during acquisition
train_test = tox21[(tox21['ms_library'].isna())                 
                   &(tox21['iris_data'].isna())                 
                   &(tox21['isabel_data'].isna())
                   &(tox21['ionization_experimental'].isna()) 
                   |(tox21['ionization_experimental']==0) 
                   ]

train_test = train_test[['InChIKey14', 'SMILES', 'sirius5_fp', 'nr.ahr', 'sr.mmp']]

tox21.shape, train_test.shape

In [ ]:
train_test

In [ ]:
expanded_fp = train_test['sirius5_fp'].apply(pd.Series)

train_test = train_test.drop(columns=['sirius5_fp']).join(expanded_fp)

train_test.head()

In [ ]:
expanded_fp.shape

In [ ]:
train_test_ahr = train_test[train_test['nr.ahr'].notna()]
train_test_mmp = train_test[train_test['sr.mmp'].notna()]

In [ ]:
train_test_ahr.shape, train_test_mmp.shape

In [ ]:
# with open('train_test_ahr.pkl', 'wb') as f:
#     pickle.dump(train_test_ahr, f)

# with open('train_test_mmp.pkl', 'wb') as f:
#     pickle.dump(train_test_mmp, f)

##### Evaluation data

In [ ]:
evaluation_data = tox21[tox21['ms_library'].notna() | 
                        tox21['iris_data'].notna() | 
                        tox21['isabel_data'].notna()|
                        tox21['ionization_experimental']==1]

In [ ]:
evaluation_data

In [ ]:
with open('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Data/MSMS/SIRIUS_output/output_mass_correction_2025-05-13/2025-05-17_experimental_pred_fp_true_fp_correct_formula.pkl', 'rb') as f:
    correct_formula = pickle.load(f)

In [ ]:
correct_formula

In [ ]:
evaluation_data_combined = pd.merge(evaluation_data, correct_formula[['InChIKey14', 'adduct','sirius5_predicted_fp']], on='InChIKey14', how = 'left')

In [ ]:
evaluation_data_combined

In [ ]:
evaluation_data_final = evaluation_data_combined[evaluation_data_combined['sirius5_predicted_fp'].notna()]

In [ ]:
evaluation_data_final

In [ ]:
evaluation_data_final_no_dupl = evaluation_data_final.drop_duplicates(subset=['sirius5_predicted_fp'], keep='first')

In [ ]:
evaluation_data_final

In [ ]:
evaluation_data_final_no_dupl

In [ ]:
evaluation_data_final_no_dupl[evaluation_data_final_no_dupl.InChIKey14.duplicated(keep=False)]

In [ ]:
evaluation_data_final_no_dupl = evaluation_data_final_no_dupl.drop_duplicates(subset=['InChIKey14', 'adduct'], keep='first')

In [ ]:
evaluation_data_final_no_dupl.shape

In [ ]:
evaluation_data_final_no_dupl = evaluation_data_final_no_dupl[['InChIKey14', 'SMILES', 'nr.ahr', 'sr.mmp', 'adduct', 'sirius5_predicted_fp']]

In [ ]:
expanded_fp = evaluation_data_final_no_dupl['sirius5_predicted_fp'].apply(pd.Series)

evaluation_data_expanded = evaluation_data_final_no_dupl.drop(columns=['sirius5_predicted_fp']).join(expanded_fp)

evaluation_data_expanded.head()

In [ ]:
evaluation_data_final_ahr = evaluation_data_expanded[evaluation_data_expanded['nr.ahr'].notna()]
evaluation_data_final_mmp = evaluation_data_expanded[evaluation_data_expanded['sr.mmp'].notna()]

In [ ]:
evaluation_data_final_ahr.shape, evaluation_data_final_mmp.shape

In [ ]:
evaluation_data_final_ahr.value_counts(subset=['nr.ahr']), evaluation_data_final_mmp.value_counts(subset=['sr.mmp'])

In [ ]:
# with open('evaluation_ahr.pkl', 'wb') as f:
#     pickle.dump(evaluation_data_final_ahr, f)
# with open('evaluation_mmp.pkl', 'wb') as f:
#     pickle.dump(evaluation_data_final_mmp, f)

## Functions for model training and testing

In [ ]:
from sklearn.utils import class_weight
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, \
                            precision_recall_curve, roc_curve, balanced_accuracy_score, \
                            precision_score, f1_score, recall_score
from sklearn.exceptions import NotFittedError
from sklearn.model_selection import KFold, cross_val_score,train_test_split 
from sklearn.metrics import roc_curve, recall_score
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from xgboost import XGBClassifier

import optuna
from optuna import create_study, logging
from optuna.pruners import MedianPruner
from optuna.integration import XGBoostPruningCallback
from sklearn.ensemble import RandomForestClassifier

In [ ]:
import warnings
warnings.filterwarnings('ignore', category = RuntimeWarning)
warnings.filterwarnings('ignore', category = UserWarning)

import pandas as pd
import numpy as np
import glob
import os
import zipfile
import shutil
import pickle
import xgboost as xgb

In [ ]:
# Code made by Ida Rahu
from sklearn.metrics import roc_curve

#Get the fpr at 90% recall
def fpr_at_recall_90(y_true, y_probs):
    '''
    Get the false positive rate at 90% recall
    '''
    fpr, tpr, thresholds = roc_curve(y_true, y_probs)
    try:
        threshold_idx = np.where(tpr >= 0.9)[0][0]
        return fpr[threshold_idx]
    except IndexError:
        return 1.0 # If we never reach 90% recall, return 1.0 (worst case)

In [ ]:
# Functions for making predictions
def MonteCarlo_prediction(fp, model, n_samples=10000):
    binary_samples = np.random.binomial(1, fp, size=(n_samples, len(fp)))
    predicted_probabilities = model.predict_proba(binary_samples)[:, 1]
    return np.mean(predicted_probabilities)

def make_predictions(data, model):
    df = data[np.concatenate([['id', 'formula'], model.feature_names_in_])]
    predicted_probabilities =  df[model.feature_names_in_].apply(lambda x: MonteCarlo_prediction(np.array(x), model=model), axis=1)
    return predicted_probabilities

### Model training: Nuclear receptor AhR

#### Data processing

In [ ]:
with open('train_test_ahr.pkl', 'rb') as f:
    train_test_ahr = pickle.load(f)

In [ ]:
# Split the data into features and target variable for AHR
X = train_test_ahr.drop(columns=['InChIKey14', 'SMILES', 'nr.ahr', 'sr.mmp'])
y = train_test_ahr[['nr.ahr']]

In [ ]:
# Divide into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=42)

In [ ]:
print(X_train.shape)

In [ ]:
# Remove features with low variance
selector = VarianceThreshold(threshold=0.1)
X_train = selector.fit_transform(X_train)
X_train = pd.DataFrame(X_train,
                       columns=X.columns[selector.get_support()])

X_test = selector.transform(X_test)
X_test = pd.DataFrame(X_test,
                       columns=X.columns[selector.get_support()])

In [ ]:
y_train = y_train.squeeze(axis=1)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
ahr_features = X_train.columns

In [ ]:
with open('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Code/Model_training/AHR/ahr_features.pkl', 'wb') as f:
    pickle.dump(ahr_features, f)

##### Preprocessing of evaluation data

In [ ]:
with open('evaluation_ahr.pkl', 'rb') as f:
    evaluation_data_ahr = pickle.load(f)

In [ ]:
evaluation_data_ahr

In [ ]:
evaluation_data_ahr.columns

In [ ]:
evaluation_ahr_fp = evaluation_data_ahr.drop(columns=['InChIKey14', 'SMILES', 'nr.ahr', 'sr.mmp', 'adduct'])
evaluation_ahr_labels = evaluation_data_ahr[['nr.ahr']]

#evaluation_ahr_fp = selector.transform(evaluation_ahr_fp)


#### Add split to tox21 dataset

In [ ]:
#Compounds which can be used for spectrum evaluation
eval_set = tox21[(tox21['ms_library'].notna())                 
                 |(tox21['iris_data'].notna())                 
                 |(tox21['isabel_data'].notna())
                 |(tox21['ionization_experimental'].notna()) 
                 &(tox21['ionization_experimental']==1)
                   ]

eval_set = eval_set[eval_set['sirius_data'].isna()] # removal of SIRIUS training data

eval_set

In [ ]:
eval_set_ahr = eval_set[eval_set['nr.ahr'].notna()]
eval_set_mmp = eval_set[eval_set['sr.mmp'].notna()]

In [ ]:
eval_set_ahr.shape, eval_set_mmp.shape

In [ ]:
tox21_added_splits = tox21 
tox21_added_splits['nr.ahr_split'] = [
    'train' if i in y_train.index else
    'test' if i in y_test.index else
    'spectrum_evaluation' if i in eval_set_ahr.index else
    np.nan
    for i in tox21_added_splits.index]


In [ ]:
tox21_added_splits

In [ ]:
tox21_added_splits[tox21_added_splits['nr.ahr_split']=='spectrum_evaluation'].shape

#### XGBoost

In [ ]:
def objective(trial):
    #Number of estimators
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)

    # Max depth of tree
    max_depth = trial.suggest_int('max_depth', 10, 100)

    # Learning rate of model
    learning_rate = trial.suggest_float('learning_rate', 0, 0.5)

    #num_boost_round = trial.suggest_int('num_boost_round', 100, 1000)

    # Minimum child weight
    min_child_weight = trial.suggest_float('min_child_weight', 0.001, 100)

    # Subsample ratio of the training instance
    subsample = trial.suggest_float('subsample', 0, 1)
    
    # Column sample by tree
    colsample_bytree = trial.suggest_float('colsample_bytree', 0, 1)

    model = XGBClassifier(n_estimators=n_estimators,
                          max_depth=max_depth,
                          learning_rate=learning_rate,
                          min_child_weight=min_child_weight,
                          subsample=subsample,
                          colsample_bytree=colsample_bytree,
                          eval_metric='logloss')
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fprs = []

    for train_idx, test_idx in skf.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[test_idx]

        model.fit(X_tr, y_tr)
        y_probs = model.predict_proba(X_val)[:, 1]
        fpr = fpr_at_recall_90(y_val, y_probs)
        fprs.append(fpr)

    return -np.mean(fprs) 

In [ ]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())

study.optimize(objective, n_trials=500)
final_params = study.best_params

model_full = XGBClassifier(**final_params,
                              random_state=0,
                              n_jobs=-1)
model_full.fit(X_train, y_train)

In [ ]:
# Save model
# with open('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Code/Model_training/AHR/XGBoost/2025-05-14_XGBoost_feat_var_0.9_ahr_CORRECT_FINAL_correct_features.pkl', 'wb') as f:
#     pickle.dump(model_full, f)

##### Model evaluation

In [ ]:
with open('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Code/Model_training/AHR/XGBoost/2025-05-14_XGBoost_feat_var_0.9_ahr_CORRECT_FINAL_correct_features.pkl', 'rb') as f:
    model_full = pickle.load(f)

In [ ]:
model_full.feature_names_in_

In [ ]:
model_full.get_params()

In [ ]:
# # Get list of feature names and save them to model
# feature_names = X_train.columns
# model_full.get_booster().feature_names = feature_names #Does this work?

# #Make file name and save model
# # file_name = f'./trained_models/nr.ahr_full.pkl'
# # pickle.dump(model_full, open(file_name, "wb"))

In [ ]:
# get threshold 
fpr_train, tpr_train, thresh_train = roc_curve(y_train, model_full.predict_proba(X_train)[:, 1])

recall_index_90 = np.argmax(tpr_train >= 0.9) 
thresh_at_90_recall_train = thresh_train[recall_index_90]


In [ ]:
roc_auc = roc_auc_score(y_test, model_full.predict_proba(X_test)[:, 1])
fpr, tpr, thresh = roc_curve(y_test, model_full.predict_proba(X_test)[:, 1],
                                     drop_intermediate=False)

recall_index_90 = np.argmax(tpr >= 0.9) 
fpr_at_90_recall = fpr[recall_index_90]
thresh_at_90_recall = thresh[recall_index_90]

#Example on how to extra the threshold for a specific recall
#thresh[recall_index_90]

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

y_pred_proba = model_full.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba >= thresh_at_90_recall).astype(int)

cm = confusion_matrix(y_test, y_pred)

disp_ahr_xgb = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Inactive', 'Active'])
disp_ahr_xgb.plot(cmap='Blues')

In [ ]:
bal_acc = balanced_accuracy_score(y_test, y_pred)

In [ ]:
bal_acc, fpr_at_90_recall, roc_auc

In [ ]:
# Save out the threshold at 90% recall
thresholds_90_recall_df = pd.DataFrame({'endpoint': ['AHR'],
                                        'model': ['XGB'],
                                        'threshold_90_recall': [thresh_at_90_recall],
                                        'fpr_90_recall': [fpr_at_90_recall],
                                        'roc_auc_score': [roc_auc],
                                        'balanced_accuracy': [bal_acc]})

In [ ]:
# # Save the ROC curve data
roc_ahr_xgb = pd.DataFrame({'fpr_ahr_xgb': fpr, 'tpr_ahr_xgb': tpr, 'thresholds_ahr_xgb': thresh})

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
recall_index_90 = np.argmax(tpr >= 0.9) 

fpr_index_90 = fpr[recall_index_90]
tpr_index_90 = tpr[recall_index_90]



In [ ]:
# Set the figure parameters
plt.rcParams.update({'figure.figsize':[9.8,9.8],
                'font.size': 16, 
                'font.weight': 'normal',
                'axes.titlesize': 12,
                'axes.labelsize': 12,
                'xtick.labelsize': 12,
                'ytick.labelsize': 12,
                'legend.fontsize': 12,
                'legend.title_fontsize': 12,
                'axes.titleweight': 'bold',
                'font.family': 'serif',
                'font.serif': ['Times New Roman'],
                'figure.dpi':300,
                
                })

fig, ax = plt.subplots(figsize=(6, 6))
ax.plot(roc_ahr_xgb['fpr_ahr_xgb'], roc_ahr_xgb['tpr_ahr_xgb'], label='AhR XGB', color='#023047', linewidth=2)

x1, y1 = fpr_index_90, 0
x2, y2 = fpr_index_90, tpr_index_90

ax.plot([x1, x2], [y1, y2], linestyle='--', color='#FB8500')

x1, y1 = 0, tpr_index_90
x2, y2 = fpr_index_90, tpr_index_90

ax.plot([x1, x2], [y1, y2], linestyle='--', color='#FB8500')

ax.set_xlabel('False Positive Rate', fontsize=20, fontweight='bold')
ax.set_ylabel('True Positive Rate', fontsize=20, fontweight='bold')

plt.text(-0.02, tpr_index_90 +0.02, f'TPR=0.9', color = '#FB8500', fontweight='bold', fontsize=20)
plt.text(fpr_index_90 + 0.01, tpr_index_90/3, f'FPR={fpr_index_90:.3f}', rotation=270, color = '#FB8500' , fontweight='bold', fontsize=20)

plt.text(0.4, 0.70, f'ROC-AUC={roc_auc:.3f}', fontweight='bold',  color='#023047', fontsize=20 )

ax.set_xticks([0,0.5,1])
ax.set_yticks([0,0.5,1])

plt.tick_params(axis='both', labelsize=18)

fig.savefig('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Visualizations/example_roc_auc.pdf', bbox_inches='tight', dpi=300, transparent=True)


##### Evaluation of probabilistic fingerprints

In [ ]:
with open('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Code/Model_training/AHR/XGBoost/2025-05-14_XGBoost_feat_var_0.9_ahr_CORRECT_FINAL_correct_features.pkl', 'rb') as f:
    model_full = pickle.load(f)

In [ ]:
with open('2025-05-18_threshold_fpr_at_90_recall_ahr_mmp_models.pkl', 'rb') as f:
    thresholds = pickle.load(f) 

In [ ]:
# Functions for making predictions
def MonteCarlo_prediction(fp, model, n_samples=10000):
    binary_samples = np.random.binomial(1, fp, size=(n_samples, len(fp)))
    predicted_probabilities = model.predict_proba(binary_samples)[:, 1]
    return np.mean(predicted_probabilities)

def make_predictions(data, model):
    model_features = [int(i) for i in model.feature_names_in_]
    df = data.iloc[:, model_features]
    predicted_probabilities =  df.apply(lambda x: MonteCarlo_prediction(np.array(x), model=model), axis=1)
    return predicted_probabilities

In [ ]:
final_predictions = evaluation_data_ahr[['nr.ahr']]

final_predictions.rename(columns={'nr.ahr': 'AHR_true_label'}, inplace=True)

In [ ]:
predictions_df = make_predictions(evaluation_ahr_fp, model_full)

thresh = thresholds[(thresholds['endpoint']=='AHR') & (thresholds['model']=='XGB')]['threshold_90_recall'].values[0]

final_predictions['AHR_XGB_proba'] = predictions_df
final_predictions['AHR_XGB'] = (predictions_df > thresh).astype(int)

In [ ]:
final_predictions

#### RF

In [ ]:
#Optuna objective for Random Forest

def objective(trial):
    # Number of trees in random forest
    n_estimators = trial.suggest_int(name="n_estimators", low=10, high=1000)

    # Number of features to consider at every split
    max_features = trial.suggest_categorical(name="max_features", choices=['log2', 'sqrt', 0.2, 0.5]) 

    # Maximum number of levels in tree
    max_depth = trial.suggest_int(name="max_depth", low=5, high=100)

    # Minimum number of samples required to split a node
    min_samples_split = trial.suggest_int(name="min_samples_split", low=2, high=20)

    # Minimum number of samples required at each leaf node
    min_samples_leaf = trial.suggest_int(name="min_samples_leaf", low=5, high=40)
    
    params = {
        "n_estimators": n_estimators,
        "max_features": max_features,
        "max_depth": max_depth,
        "min_samples_split": min_samples_split,
        "min_samples_leaf": min_samples_leaf
    }
    model = RandomForestClassifier(random_state=101, **params)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fprs = []

    for train_idx, test_idx in skf.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[test_idx]

        model.fit(X_tr, y_tr)
        y_probs = model.predict_proba(X_val)[:, 1]
        fpr = fpr_at_recall_90(y_val, y_probs)
        fprs.append(fpr)
    
    return -np.mean(fprs)

In [ ]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())
study.optimize(objective, n_trials=500)
final_params = study.best_params

model_full = RandomForestClassifier(**final_params,
                                    random_state=0,
                                    n_jobs=-1)
model_full.fit(X_train, y_train)

In [ ]:
# with open('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Code/Model_training/AHR/RF/2025-05-14_RF_feat_var_0.9_ahr_CORRECT_FINAL_correct_features.pkl', 'wb') as f:
#     pickle.dump(model_full, f)

##### Model evaluation

In [ ]:
with open('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Code/Model_training/AHR/RF/2025-05-14_RF_feat_var_0.9_ahr_CORRECT_FINAL_correct_features.pkl', 'rb') as f:
    model_full = pickle.load(f)

In [ ]:
model_full.get_params()

In [ ]:
model_full.score(X_test, y_test)


In [ ]:
roc_auc = roc_auc_score(y_test, model_full.predict_proba(X_test)[:, 1])
fpr, tpr, thresh = roc_curve(y_test, model_full.predict_proba(X_test)[:, 1],
                                     drop_intermediate=False)

recall_index_90 = np.argmax(tpr >= 0.9) 
fpr_at_90_recall = fpr[recall_index_90]
thresh_at_90_recall = thresh[recall_index_90]

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

#adjust the probability threshold to get 90% recall
y_pred_proba = model_full.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba >= thresh_at_90_recall).astype(int)

cm = confusion_matrix(y_test, y_pred)

disp_ahr_rf = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Inactive', 'Active'])
disp_ahr_rf.plot(cmap='Blues')

bal_acc = balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Save out the threshold at 90% recall
thresholds_90_recall = pd.DataFrame({'endpoint': ['AHR'],
                                        'model': ['RF'],
                                        'threshold_90_recall': [thresh_at_90_recall],
                                        'fpr_90_recall': [fpr_at_90_recall],
                                        'roc_auc_score': [roc_auc],
                                        'balanced_accuracy': [bal_acc]})

thresholds_90_recall_df = pd.concat([thresholds_90_recall_df, thresholds_90_recall], ignore_index=True)

In [ ]:
# # Save the ROC curve data
roc_ahr_rf = pd.DataFrame({'fpr_ahr_rf': fpr, 'tpr_ahr_rf': tpr, 'thresholds_ahr_rf': thresh})

##### Evaluation of probabilistic fingerprints

In [ ]:
with open('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Code/Model_training/AHR/RF/2025-05-14_RF_feat_var_0.9_ahr_CORRECT_FINAL_correct_features.pkl', 'rb') as f:
    model_full = pickle.load(f)

In [ ]:
# Functions for making predictions
def MonteCarlo_prediction(fp, model, n_samples=10000):
    binary_samples = np.random.binomial(1, fp, size=(n_samples, len(fp)))
    predicted_probabilities = model.predict_proba(binary_samples)[:, 1]
    return np.mean(predicted_probabilities)

def make_predictions(data, model, feature_names):
    df = data.iloc[:, list(feature_names)]
    predicted_probabilities =  df.apply(lambda x: MonteCarlo_prediction(np.array(x), model=model), axis=1)
    return predicted_probabilities

In [ ]:
thresh

In [ ]:
predictions_df = make_predictions(evaluation_ahr_fp, model_full, ahr_features)

thresh = thresholds[(thresholds['endpoint']=='AHR') & (thresholds['model']=='RF')]['threshold_90_recall'].values[0]


final_predictions['AHR_RF_proba'] = predictions_df
final_predictions['AHR_RF'] = (predictions_df > thresh).astype(int)

In [ ]:
final_predictions

In [ ]:
with open('final_predictions_experimental_ahr.pkl', 'wb') as f:
    pickle.dump(final_predictions, f)

### Model training: Stress response MMP

#### Preprocessing of data

In [ ]:
import pickle
import pandas as pd
import numpy as np
import sklearn

from sklearn.utils import class_weight
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, \
                            precision_recall_curve, roc_curve, balanced_accuracy_score, \
                            precision_score, f1_score, recall_score
from sklearn.exceptions import NotFittedError
from sklearn.model_selection import KFold, cross_val_score,train_test_split 
from sklearn.metrics import roc_curve, recall_score
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from xgboost import XGBClassifier

import optuna
from optuna import create_study, logging
from optuna.pruners import MedianPruner
from optuna.integration import XGBoostPruningCallback
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import root_mean_squared_error,mean_absolute_error
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import scale

In [ ]:
with open('train_test_mmp.pkl', 'rb') as f:
    train_test_mmp = pickle.load(f)

In [ ]:
train_test_mmp.shape

In [ ]:
# Split the data into features and target variable for AHR
X = train_test_mmp.drop(columns=['InChIKey14', 'SMILES', 'nr.ahr', 'sr.mmp'])
y = train_test_mmp['sr.mmp']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=42)

In [ ]:
print(X_train.shape)

In [ ]:
# Remove features with low variance
selector = VarianceThreshold(threshold=0.1)
X_train = selector.fit_transform(X_train)
X_train = pd.DataFrame(X_train,
                       columns=X.columns[selector.get_support()])

X_test = selector.transform(X_test)
X_test = pd.DataFrame(X_test,
                       columns=X.columns[selector.get_support()])

In [ ]:
print(X_train.shape)

In [ ]:
print(X_test.shape)

In [ ]:
mmp_features = X_train.columns

In [ ]:
with open('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Code/Model_training/MMP/mmp_features.pkl', 'wb') as f:
    pickle.dump(mmp_features, f)

##### Preprocessing of evaluation data

In [ ]:
with open('evaluation_mmp.pkl', 'rb') as f:
    evaluation_data_mmp = pickle.load(f)

In [ ]:
evaluation_mmp_fp = evaluation_data_mmp.drop(columns=['InChIKey14', 'SMILES', 'nr.ahr', 'sr.mmp', 'adduct'])
evaluation_mmp_labels = evaluation_data_mmp[['sr.mmp']]

#### Add split to tox21 dataset

In [ ]:
# Add the sr.mmp split
tox21_added_splits['sr.mmp_split'] = [
    'train' if i in y_train.index else
    'test' if i in y_test.index else
    'spectrum_evaluation' if i in eval_set_mmp.index else
    np.nan
    for i in tox21_added_splits.index]


In [ ]:
tox21_added_splits

In [ ]:
tox21_added_splits[tox21_added_splits['sr.mmp_split']=='train'].shape

In [ ]:
# with open('2025-05-12_tox21_ahr_mmp_6999_compounds_apci.pkl', 'wb') as f:
#     pickle.dump(tox21_added_splits, f)

#### RF

In [ ]:
#Optuna objective for Random Forest

def objective(trial):
    # Number of trees in random forest
    n_estimators = trial.suggest_int(name="n_estimators", low=10, high=1000)

    # Number of features to consider at every split
    max_features = trial.suggest_categorical(name="max_features", choices=['log2', 'sqrt', 0.2, 0.5]) 

    # Maximum number of levels in tree
    max_depth = trial.suggest_int(name="max_depth", low=5, high=100)

    # Minimum number of samples required to split a node
    min_samples_split = trial.suggest_int(name="min_samples_split", low=2, high=20)

    # Minimum number of samples required at each leaf node
    min_samples_leaf = trial.suggest_int(name="min_samples_leaf", low=5, high=40)
    
    params = {
        "n_estimators": n_estimators,
        "max_features": max_features,
        "max_depth": max_depth,
        "min_samples_split": min_samples_split,
        "min_samples_leaf": min_samples_leaf
    }
    model = RandomForestClassifier(random_state=101, **params)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fprs = []

    for train_idx, test_idx in skf.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[test_idx]

        model.fit(X_tr, y_tr)
        y_probs = model.predict_proba(X_val)[:, 1]
        fpr = fpr_at_recall_90(y_val, y_probs)
        fprs.append(fpr)
    
    return -np.mean(fprs)

In [ ]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())
study.optimize(objective, n_trials=500)
final_params = study.best_params

model_full = RandomForestClassifier(**final_params,
                              random_state=0,
                              n_jobs=-1)
model_full.fit(X_train, y_train)

In [ ]:
# with open('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Code/Model_training/MMP/RF/2025-05-14_RF_feat_var_0.9_mmp_CORRECT_FINAL_correct_features.pkl', 'wb') as f:
#     pickle.dump(model_full, f)

##### Model evaulation

In [ ]:
with open('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Code/Model_training/MMP/RF/2025-05-14_RF_feat_var_0.9_mmp_CORRECT_FINAL_correct_features.pkl', 'rb') as f:
    model_full = pickle.load(f)

In [ ]:
model_full.get_params()

In [ ]:
model_full.score(X_test, y_test)

In [ ]:
roc_auc = roc_auc_score(y_test, model_full.predict_proba(X_test)[:, 1])
fpr, tpr, thresh = roc_curve(y_test, model_full.predict_proba(X_test)[:, 1],
                                     drop_intermediate=False)

recall_index_90 = np.argmax(tpr >= 0.9) 
fpr_at_90_recall = fpr[recall_index_90]
thresh_at_90_recall = thresh[recall_index_90]

fpr_at_90_recall, thresh_at_90_recall

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

#adjust the probability threshold to get 90% recall
y_pred_proba = model_full.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba >= thresh_at_90_recall).astype(int)

cm = confusion_matrix(y_test, y_pred)

disp_mmp_rf = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Inactive', 'Active'])
disp_mmp_rf.plot(cmap='Blues')

bal_acc = balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Save out the threshold at 90% recall
thresholds_90_recall = pd.DataFrame({'endpoint': ['MMP'],
                                        'model': ['RF'],
                                        'threshold_90_recall': [thresh_at_90_recall],
                                        'fpr_90_recall': [fpr_at_90_recall],
                                        'roc_auc_score': [roc_auc],
                                        'balanced_accuracy': [bal_acc]})

thresholds_90_recall_df = pd.concat([thresholds_90_recall_df, thresholds_90_recall], ignore_index=True)

In [ ]:
thresholds_90_recall

In [ ]:
# # Save the ROC curve data
roc_mmp_rf = pd.DataFrame({'fpr_mmp_rf': fpr, 'tpr_mmp_rf': tpr, 'thresholds_mmp_rf': thresh})

##### Evaluation of probabilistic fingerprints

In [ ]:
with open('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Code/Model_training/MMP/RF/2025-05-14_RF_feat_var_0.9_mmp_CORRECT_FINAL_correct_features.pkl', 'rb') as f:
    model_full = pickle.load(f)

In [ ]:
# Functions for making predictions
def MonteCarlo_prediction(fp, model, n_samples=10000):
    binary_samples = np.random.binomial(1, fp, size=(n_samples, len(fp)))
    predicted_probabilities = model.predict_proba(binary_samples)[:, 1]
    return np.mean(predicted_probabilities)

def make_predictions(data, model, feature_names):
    df = data.iloc[:, list(feature_names)]
    predicted_probabilities =  df.apply(lambda x: MonteCarlo_prediction(np.array(x), model=model), axis=1)
    return predicted_probabilities

In [ ]:
final_predictions = evaluation_data_mmp[['sr.mmp']]

final_predictions.rename(columns={'sr.mmp': 'MMP_true_label'}, inplace=True)

In [ ]:
predictions_df = make_predictions(evaluation_mmp_fp, model_full, mmp_features)

thresh = thresholds[(thresholds['endpoint']=='MMP') & (thresholds['model']=='RF')]['threshold_90_recall'].values[0]


final_predictions['MMP_RF_proba'] = predictions_df
final_predictions['MMP_RF'] = (predictions_df > thresh).astype(int)

In [ ]:
final_predictions

#### XGBoost

In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 10, 100)
    learning_rate = trial.suggest_float('learning_rate', 0, 0.5)
    num_boost_round = trial.suggest_int('num_boost_round', 100, 1000)
    min_child_weight = trial.suggest_float('min_child_weight', 0.001, 100)
    subsample = trial.suggest_float('subsample', 0, 1)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0, 1)

    model = XGBClassifier(n_estimators=n_estimators,
                          max_depth=max_depth,
                          learning_rate=learning_rate,
                          min_child_weight=min_child_weight,
                          subsample=subsample,
                          colsample_bytree=colsample_bytree,
                          eval_metric='logloss')
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fprs = []

    for train_idx, test_idx in skf.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[test_idx]

        model.fit(X_tr, y_tr)
        y_probs = model.predict_proba(X_val)[:, 1]
        fpr = fpr_at_recall_90(y_val, y_probs)
        fprs.append(fpr)

    return -np.mean(fprs) 

In [ ]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())

study.optimize(objective, n_trials=500)
final_params = study.best_params

model_full = XGBClassifier(**final_params,
                              random_state=0,
                              n_jobs=-1)
model_full.fit(X_train, y_train)

In [ ]:
# with open('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Code/Model_training/MMP/XGBoost/2025-05-14_XGBoost_feat_var_0.9_mmp_CORRECT_FINAL_correct_features.pkl', 'wb') as f:
#     pickle.dump(model_full, f)

##### Model evaulation

In [ ]:
with open('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Code/Model_training/MMP/XGBoost/2025-05-14_XGBoost_feat_var_0.9_mmp_CORRECT_FINAL_correct_features.pkl', 'rb') as f:
    model_full = pickle.load(f)

In [ ]:
model_full.get_params()

In [ ]:
model_full.score(X_test, y_test)


In [ ]:
roc_auc = roc_auc_score(y_test, model_full.predict_proba(X_test)[:, 1])
fpr, tpr, thresh = roc_curve(y_test, model_full.predict_proba(X_test)[:, 1],
                                     drop_intermediate=False)

recall_index_90 = np.argmax(tpr >= 0.9) 
fpr_at_90_recall = fpr[recall_index_90]
thresh_at_90_recall = thresh[recall_index_90]

fpr_at_90_recall, thresh_at_90_recall

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

#adjust the probability threshold to get 90% recall
y_pred_proba = model_full.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba >= thresh_at_90_recall).astype(int)

cm = confusion_matrix(y_test, y_pred)

disp_mmp_xgb = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Inactive', 'Active'])
disp_mmp_xgb.plot(cmap='Blues')

bal_acc = balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Save out the threshold at 90% recall
thresholds_90_recall = pd.DataFrame({'endpoint': ['MMP'],
                                        'model': ['XGB'],
                                        'threshold_90_recall': [thresh_at_90_recall],
                                        'fpr_90_recall': [fpr_at_90_recall],
                                        'roc_auc_score': [roc_auc],
                                        'balanced_accuracy': [bal_acc]})

thresholds_90_recall_df = pd.concat([thresholds_90_recall_df, thresholds_90_recall], ignore_index=True)

In [ ]:
# # Save the ROC curve data
roc_mmp_xgb = pd.DataFrame({'fpr_mmp_xgb': fpr, 'tpr_mmp_xgb': tpr, 'thresholds_mmp_xgb': thresh})

##### Evaluation of probabilistic fingerprints

In [ ]:
# Functions for making predictions
def MonteCarlo_prediction(fp, model, n_samples=10000):
    binary_samples = np.random.binomial(1, fp, size=(n_samples, len(fp)))
    predicted_probabilities = model.predict_proba(binary_samples)[:, 1]
    return np.mean(predicted_probabilities)

def make_predictions(data, model, feature_names):
    df = data.iloc[:, list(feature_names)]
    predicted_probabilities =  df.apply(lambda x: MonteCarlo_prediction(np.array(x), model=model), axis=1)
    return predicted_probabilities

In [ ]:
thresholds

In [ ]:
predictions_df = make_predictions(evaluation_mmp_fp, model_full, mmp_features)

thresh = thresholds[(thresholds['endpoint']=='MMP') & (thresholds['model']=='XGB')]['threshold_90_recall'].values[0]

final_predictions['MMP_XGB_proba'] = predictions_df
final_predictions['MMP_XGB'] = (predictions_df > thresh).astype(int)

In [ ]:
final_predictions

In [ ]:
with open('final_predictions_experimental_mmp.pkl', 'wb') as f:
    pickle.dump(final_predictions, f)

# Visualization

## Confusion matrices

In [ ]:
plt.rcParams.keys()

In [ ]:
import matplotlib.pyplot as plt

# Set the figure parameters
plt.rcParams.update({'figure.figsize':[9.8,9.8],
                'font.size': 16, 
                'font.weight': 'normal',
                'axes.titlesize': 12,
                'axes.labelsize': 12,
                'xtick.labelsize': 12,
                'ytick.labelsize': 12,
                'legend.fontsize': 12,
                'legend.title_fontsize': 12,
                'axes.titleweight': 'bold',
                'font.family': 'serif',
                'font.serif': ['Times New Roman'],
                'figure.dpi':300,
                
                })

fig, ax = plt.subplots(2, 2, figsize=(10, 9.5), dpi=300)
# AHR
disp_ahr_rf.plot(ax=ax[0, 0], cmap='Blues', colorbar=False)
disp_ahr_rf.ax_.set_title('AhR RF')
disp_ahr_rf.ax_.set_aspect('equal')

disp_ahr_xgb.plot(ax=ax[0, 1], cmap='Blues', colorbar=False)
disp_ahr_xgb.ax_.set_title('AhR XGB')
disp_ahr_xgb.ax_.set_aspect('equal')

# MMP
disp_mmp_rf.plot(ax=ax[1, 0], cmap='Oranges', colorbar=False)
disp_mmp_rf.ax_.set_title('MMP RF')
disp_mmp_rf.ax_.set_aspect('equal')

disp_mmp_xgb.plot(ax=ax[1, 1], cmap='Oranges', colorbar=False)
disp_mmp_xgb.ax_.set_title('MMP XGB')
disp_mmp_xgb.ax_.set_aspect('equal')

plt.tight_layout()

plt.suptitle('Intermediate test set\nConfusion matrices for AhR and MMP models at TPR=90%', fontsize=16, fontweight='bold', y=1.03)



plt.savefig('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Visualizations/2025-05-18_confusion_matrices_ahr_mmp_new.pdf', dpi=300, bbox_inches='tight')


## ROC-AUC

In [ ]:
#Concatenate all ROC data
roc_data = pd.concat([roc_ahr_rf, roc_ahr_xgb, roc_mmp_rf, roc_mmp_xgb], axis=1)

In [ ]:
#Plot all ROC curves
fig, ax = plt.subplots(1, 2, figsize=(8, 5), dpi=300, tight_layout=True, sharex=True, sharey=True)

ax[0].plot(roc_data['fpr_ahr_rf'], roc_data['tpr_ahr_rf'], label='AhR RF', color='#219EBC')
ax[0].plot(roc_data['fpr_ahr_xgb'], roc_data['tpr_ahr_xgb'], label='AhR XGB', color='#023047')
ax[0].grid(visible=True, which='both', linewidth=0.5, alpha=0.5)
ax[0].set_title('AhR models')
ax[0].legend(loc='lower right')
ax[0].set_aspect('equal')

ax[1].plot(roc_data['fpr_mmp_rf'], roc_data['tpr_mmp_rf'], label='MMP RF', color='#FFB703')
ax[1].plot(roc_data['fpr_mmp_xgb'], roc_data['tpr_mmp_xgb'], label='MMP XGB', color='#FB8500')
ax[1].grid(visible=True, which='both', linewidth=0.5, alpha=0.5)
ax[1].set_title('MMP models')
ax[1].legend(loc='lower right')
ax[1].set_aspect('equal')


#fig.suptitle('Intermediate test set\nROC-AUC for AhR and MMP models', fontweight='bold', fontsize=16, y = 0.92)
fig.supxlabel('False Positive Rate', y=0.12, fontsize=12)
fig.supylabel('True Positive Rate', x=0.05, fontsize=12)

fig.savefig('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Visualizations/2025-05-26_roc_curves_ahr_mmp_new.pdf', dpi=300, bbox_inches='tight',
            transparent=True)


# FPR rate

In [ ]:
thresholds_90_recall_df

In [ ]:
with open('2025-05-18_threshold_fpr_at_90_recall_ahr_mmp_models.pkl', 'wb') as f:
    pickle.dump(thresholds_90_recall_df, f)

# From OPTUNA code by Anneli

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

#Fit RF classifier model without any hyperparameter tuning
rf.fit(X_train, y_train)

#Predict on the test set
y_pred = rf.predict(X_test)

#Determine the RMSE and MAE for the test set
test_rmse = root_mean_squared_error(y_test, y_pred)
test_mae = mean_absolute_error(y_test, y_pred)
print(f'Test set RMSE: {test_rmse} \n test set MAE: {test_mae}')

# My own code for Confusion matrix (not done)

In [ ]:
#Confusion matrix of the results
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import matplotlib.pyplot as plt
import seaborn as sns


#Create confusion matrix
cm = confusion_matrix(y_test, y_pred)


# Step 2: Plot using seaborn heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)

plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

# Idas code

In [ ]:
import warnings
warnings.filterwarnings('ignore', category = RuntimeWarning)

import sys
import pandas as pd
import numpy as np
import timeit

import matplotlib.pyplot as plt
import seaborn as sns 

import pickle


from sklearn.utils import class_weight
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, \
                            precision_recall_curve, roc_curve, balanced_accuracy_score, \
                            precision_score, f1_score, recall_score
from sklearn.exceptions import NotFittedError
from sklearn.model_selection import KFold, cross_val_score,train_test_split 
from sklearn.metrics import roc_curve, recall_score
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from xgboost import XGBClassifier

import optuna
from optuna import create_study, logging
from optuna.pruners import MedianPruner
from optuna.integration import XGBoostPruningCallback

In [ ]:
def fpr_at_recall_90(y_true, y_probs):
    fpr, tpr, thresholds = roc_curve(y_true, y_probs)
    try:
        threshold_idx = np.where(tpr >= 0.9)[0][0]
        return fpr[threshold_idx]
    except IndexError:
        return 1.0 

In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 10, 100)
    learning_rate = trial.suggest_float('learning_rate', 0, 0.5)
    num_boost_round = trial.suggest_int('num_boost_round', 100, 1000)
    min_child_weight = trial.suggest_float('min_child_weight', 0.001, 100)
    subsample = trial.suggest_float('subsample', 0, 1)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0, 1)

    model = XGBClassifier(n_estimators=n_estimators,
                          max_depth=max_depth,
                          learning_rate=learning_rate,
                          min_child_weight=min_child_weight,
                          subsample=subsample,
                          colsample_bytree=colsample_bytree,
                          eval_metric='logloss')
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fprs = []

    for train_idx, test_idx in skf.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[test_idx]

        model.fit(X_tr, y_tr)
        y_probs = model.predict_proba(X_val)[:, 1]
        fpr = fpr_at_recall_90(y_val, y_probs)
        fprs.append(fpr)

    return -np.mean(fprs) 

In [ ]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())

study.optimize(objective, n_trials=500)
final_params = study.best_params

model_full = XGBClassifier(**final_params,
                              random_state=0,
                              n_jobs=-1)
model_full.fit(X_train, y_train)

In [ ]:
model_full.get_booster().feature_names = feature_names
file_name = f'./trained_models/nr.ahr_full.pkl'
pickle.dump(model_full, open(file_name, "wb"))

In [ ]:
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
fpr, tpr, thresh = roc_curve(y_test, model.predict_proba(X_test)[:, 1],
                                     drop_intermediate=False)

recall_index_90 = np.argmax(tpr >= 0.9) 
fpr_at_90_recall = fpr[recall_index_90]

#Example on how to extra the threshold for a specific recall 
#thresh[recall_index_90]